In [1]:
import numpy as np
import plotly.graph_objects as go
from tqdm.auto import trange

from lda import lda
from lda.data.corpus import Corpus
from lda.data.document import Document
from lda.data.word import Word
import lda.utils as utils

In [2]:
please = Word('Please', 'please', include=True)
crash = Word('crash', 'crash', include=True)
grandma = Word('grandma', 'grandma', include=True)
sun = Word('sun', 'sun', include=True)
doc1 = Document(
    words=[
        please,
        Word('do', 'do', include=False),
        Word('not', 'not', include=False),
        crash,
        Word('.', '.', include=False),
    ]
)
doc2 = Document(
    words=[
        please,
        Word('do', 'do', include=False),
        Word('not', 'not', include=False),
        grandma,
        Word('.', '.', include=False),
    ]
)
doc3 = Document(
    words=[
        grandma,
        grandma,
        sun,
    ]
)
corpus = Corpus(documents=[doc1, doc2, doc3])
corpus

Corpus with 3 documents

In [3]:
def assert_numerically_ok(array):
    assert np.all(~np.isnan(array))
    assert np.all(~np.isinf(array))

In [4]:
def assert_lda(seed):
    with utils.np_seed(seed): # 62 is the one that breaks
        params, lower_bound_evol = lda(corpus, num_topics=2, num_iterations=32)
    assert set(params.keys()) == {'alpha', 'beta', 'phis', 'gammas'}

    assert params['alpha'].shape == (2,)
    assert_numerically_ok(params['alpha'])
    assert np.all(params['alpha'] > 0)

    assert len(params['beta']) == 2
    for beta_row in params['beta']:
        assert len(beta_row) == 4
        assert_numerically_ok(list(beta_row.values()))
        assert set(beta_row.keys()) == {please, crash, grandma, sun}
        assert np.isclose(sum(beta_row.values()), 1)

    assert len(params['phis']) == 3
    assert set(params['phis'].keys()) == {doc1, doc2, doc3}
    for document, phi_row in params['phis'].items():
        assert phi_row.shape == (len(document), 2)
        assert_numerically_ok(phi_row)
        assert np.all(np.isclose(phi_row.sum(axis=1), 1))

    assert len(params['gammas']) == 3
    assert set(params['gammas'].keys()) == {doc1, doc2, doc3}
    for document, gamma_row in params['gammas'].items():
        assert gamma_row.shape == (2,)
        assert_numerically_ok(gamma_row)
        assert np.all(gamma_row > 0)
        assert np.all(gamma_row >= params['alpha'])

    assert np.all(lower_bound_evol[1:] - lower_bound_evol[:-1] >= 0)

In [5]:
assert_lda(9)

{'alpha': array([0.01037415, 0.50187459]),
 'beta': [{Word(original_form='grandma', lda_form='grandma', include=True): 0.1435070382328889,
           Word(original_form='Please', lda_form='please', include=True): 0.5006432587935381,
           Word(original_form='sun', lda_form='sun', include=True): 0.2207055707081681,
           Word(original_form='crash', lda_form='crash', include=True): 0.13514413226540492},
          {Word(original_form='grandma', lda_form='grandma', include=True): 0.07668503255089015,
           Word(original_form='Please', lda_form='please', include=True): 0.3817921321387416,
           Word(original_form='sun', lda_form='sun', include=True): 0.31518777548144816,
           Word(original_form='crash', lda_form='crash', include=True): 0.22633505982892008}],
 'gammas': {Document including 3 words: array([0.01037418, 3.50187457]),
            Document including 2 words: array([0.01037416, 2.50187458]),
            Document including 2 words: array([0.01037415, 2.501

  0%|          | 0/32 [00:00<?, ?it/s]

Document including 2 words lower bound is -3.20519034948164
Document including 2 words lower bound is -2.1065780608135043
Document including 3 words lower bound is -3.6492022858869575
Document including 2 words lower bound is -3.2024425367127094
Document including 2 words lower bound is -2.103830248044602
Document including 3 words lower bound is -3.6457468469985486
Document including 2 words lower bound is -3.201326080565906
Document including 2 words lower bound is -2.1027137918977985
Document including 3 words lower bound is -3.6442680097626834
Document including 2 words lower bound is -3.2007195706430878
Document including 2 words lower bound is -2.1021072819749804
Document including 3 words lower bound is -3.6434417652668003
Document including 2 words lower bound is -3.2003385232692096
Document including 2 words lower bound is -2.1017262346011023
Document including 3 words lower bound is -3.6429134076693543
Document including 2 words lower bound is -3.2000769456876412
Document inc

In [6]:
for seed in trange(100):
    assert_lda(seed)

  0%|          | 0/100 [00:00<?, ?it/s]

{'alpha': array([0.5488135 , 0.71518937]),
 'beta': [{Word(original_form='grandma', lda_form='grandma', include=True): 0.19107688284120386,
           Word(original_form='Please', lda_form='please', include=True): 0.2718584733852522,
           Word(original_form='sun', lda_form='sun', include=True): 0.29131130808112,
           Word(original_form='crash', lda_form='crash', include=True): 0.24575333569242383},
          {Word(original_form='grandma', lda_form='grandma', include=True): 0.3600506427818107,
           Word(original_form='Please', lda_form='please', include=True): 0.16349449532149288,
           Word(original_form='sun', lda_form='sun', include=True): 0.14326419052519523,
           Word(original_form='crash', lda_form='crash', include=True): 0.3331906713715011}],
 'gammas': {Document including 3 words: array([1.14408964, 3.11991323]),
            Document including 2 words: array([1.45520026, 1.80880261]),
            Document including 2 words: array([1.14956005, 2.11444

  0%|          | 0/32 [00:00<?, ?it/s]

Document including 2 words lower bound is -3.407267263475272
Document including 2 words lower bound is -2.367020032270053
Document including 3 words lower bound is -3.942798090043405
Document including 2 words lower bound is -3.29854405824643
Document including 2 words lower bound is -2.2871997718840427
Document including 3 words lower bound is -3.793567973717638
Document including 2 words lower bound is -3.2448566836414052
Document including 2 words lower bound is -2.2543742294609865
Document including 3 words lower bound is -3.698195821389484
Document including 2 words lower bound is -3.2143607063800324
Document including 2 words lower bound is -2.2395282048322755
Document including 3 words lower bound is -3.6255129367959182
Document including 2 words lower bound is -3.1953598536507934
Document including 2 words lower bound is -2.233500212021612
Document including 3 words lower bound is -3.5668243885275404
Document including 2 words lower bound is -3.1815754886849597
Document includi

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

Document including 2 words lower bound is -3.2106288833599237
Document including 2 words lower bound is -2.1120165946918092
Document including 3 words lower bound is -3.656577043615883
Document including 2 words lower bound is -3.2058400972731746
Document including 2 words lower bound is -2.10722780860506
Document including 3 words lower bound is -3.6505022367481246
Document including 2 words lower bound is -3.203792087355936
Document including 2 words lower bound is -2.1051797986878213
Document including 3 words lower bound is -3.6477777040447066
Document including 2 words lower bound is -3.202653785193341
Document including 2 words lower bound is -2.1040414965252197
Document including 3 words lower bound is -3.6462229613804595
Document including 2 words lower bound is -3.2019291522548556
Document including 2 words lower bound is -2.103316863586741
Document including 3 words lower bound is -3.645216441722397
Document including 2 words lower bound is -3.2014274126166242
Document includ

  0%|          | 0/32 [00:00<?, ?it/s]

Document including 2 words lower bound is nan
Document including 2 words lower bound is nan
Document including 3 words lower bound is nan
Document including 2 words lower bound is nan
Document including 2 words lower bound is nan
Document including 3 words lower bound is nan


AssertionError: 